<a href="https://colab.research.google.com/github/Fabian-Urbina/sentyment_analysis_chilean_subreddits/blob/main/sentyment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Scrap From Reddit

In [ ]:
%pip install praw
import praw
import warnings
import logging
warnings.filterwarnings("ignore", category=UserWarning, module='praw')

# Silencia los warnings de PRAW
logging.getLogger("praw").setLevel(logging.ERROR)
# -----------------------------
# Configurar Reddit
# -----------------------------
reddit = praw.Reddit(
    client_id='3hMnYve-0PzsKXPpMBjZwg',
    client_secret='08j3U0tOSRL0T2rUdovI7zEsxEFzDw',
    user_agent='scrapper'
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 15.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd

subreddits = ['chile', 'Republicadechile','Santiago','Chileit','Chilegames']
comments_data = []
for subreddit_name in subreddits:
    subreddit = reddit.subreddit(subreddit_name)
    for post in subreddit.hot(limit=100):  # puedes ajustar el número de posts
        post.comments.replace_more(limit=0)  # para traer todos los comentarios sin "MoreComments"
        for comment in post.comments.list():
            comments_data.append({
                'subreddit':subreddit_name,'comment':comment.body
                })

# -----------------------------
# Guardar en CSV
# -----------------------------
df=pd.DataFrame(comments_data)
df.to_csv('comments.csv', index=False)
from google.colab import files
files.download('/content/comments.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import re

#Puedes comenzar aqui si descargas el dataset
#df = pd.read_csv('comments.csv') # poner ruta al dataset de github
#print("Comentarios iniciales:", len(df))
#print(df.sample(5))

def clean_comment(text):
    if pd.isna(text):
        return ""
    # Eliminar URLs (http, https, www)
    text = re.sub(r"http\S+|www\.\S+", "", text)
    return text.strip()

# 1. Eliminar los que están marcados como borrados
mask = ~df['comment'].str.lower().isin(["[deleted]", "[removed]"])
df = df[mask].copy()

# 2. Limpiar URLs
df['comment'] = df['comment'].apply(clean_comment)

# 3. (Opcional) Eliminar filas vacías después de limpiar
df = df[df['comment'].str.strip() != ""]

print("Comentarios finales:", len(df))
print(df.sample(5))


Comentarios finales: 13444
             subreddit                                            comment
7440  Republicadechile  Tiene el 30% asegurado. Los amigos del Boris. ...
3508             chile  Eso es bueno. Si la elección es mañana, gana k...
4397             chile  Abogado: el OS10 no se requiere para labores d...
3895             chile        Pero por los lentes talves esta medio ciega
5610  Republicadechile  Invítala a un panorama que a ti te entretenga,...


In [ ]:
import pandas as pd
from transformers import AutoTokenizer, pipeline
import torch

# Modelo y tokenizer
model_name = "finiteautomata/beto-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(model_name)
device = 0 if torch.cuda.is_available() else -1

sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model=model_name,
    device=device
)

# Función para dividir en chunks
def chunk_text(text, max_tokens=500):
    tokens = tokenizer.encode(text, add_special_tokens=False)
    chunks = []
    for i in range(0, len(tokens), max_tokens):
        chunk_ids = tokens[i:i+max_tokens]
        chunk_text = tokenizer.decode(chunk_ids, skip_special_tokens=True)
        chunks.append(chunk_text)
    return chunks

# Función para obtener el sentimiento promedio de un comentario
def get_comment_sentiment_avg(comment):
    chunks = chunk_text(comment, max_tokens=500)
    chunk_sentiments = [sentiment_pipeline(c, truncation=True, max_length=512)[0] for c in chunks]

    # Calcular promedio de scores por label
    label_scores = {}
    for s in chunk_sentiments:
        label = s['label']
        score = s['score']
        label_scores[label] = label_scores.get(label, []) + [score]

    # Promedio por label
    avg_scores = {label: sum(scores)/len(scores) for label, scores in label_scores.items()}

    # Elegir la etiqueta con mayor promedio
    best_label = max(avg_scores, key=lambda x: avg_scores[x])
    best_score = avg_scores[best_label]

    return {
    'label': best_label,
    'score': best_score,
    'score_POS': avg_scores.get('POS', 0),
    'score_NEG': avg_scores.get('NEG', 0),
    'score_NEU': avg_scores.get('NEU', 0)
    }

# Aplicar al dataframe
df['sentiment_info'] = df['comment'].apply(get_comment_sentiment_avg)
df['sentiment_label'] = df['sentiment_info'].apply(lambda x: x['label'])
df['sentiment_score'] = df['sentiment_info'].apply(lambda x: x['score'])
df['score_POS'] = df['sentiment_info'].apply(lambda x: x['score_POS'])
df['score_NEG'] = df['sentiment_info'].apply(lambda x: x['score_NEG'])
df['score_NEU'] = df['sentiment_info'].apply(lambda x: x['score_NEU'])




Device set to use cuda:0
Token indices sequence length is longer than the specified maximum sequence length for this model (612 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
df.head()
df.to_csv('comments.csv', index=True)
from google.colab import files
files.download('/content/comments.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df['sentiment_label'].value_counts(normalize=True)

,proportion
sentiment_label,
NEU,0.517777
NEG,0.383442
POS,0.098780


In [ ]:
df2=df.where(df['sentiment_score']<0.55).dropna()[['comment','sentiment_label']]
df2.to_csv('ambiguous_comments.csv', index=True)
from google.colab import files
files.download('/content/ambiguous_comments.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>